In [2]:
pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install quantile-forest

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
from pathlib import Path

root = Path().resolve() / "topquartile"
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print("Added to PythonPath", root)

Added to PythonPath C:\Users\LO79RS\topquartile\topquartile


In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


In [6]:
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import TechnicalCovariateTransform
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform


In [7]:
# Defining Dataloader with covariates_may2025v2
covtrans_config = [(
TechnicalCovariateTransform,
dict(sma=[20, 30], ema=[20, 30], momentum_change=True, volatility=[20, 30]),
)]

labeltrans_config = [(BinaryLabelTransform, dict(label_duration=20, quantile=0.75))]

partition_config = dict(n_splits=5, gap=20, max_train_size=504, test_size=60)

dataloader = DataLoader(
data_id="covariates_may2025v2",
covariate_transform=covtrans_config,
label_transform=labeltrans_config,
partition_class=PurgedTimeSeriesPartition,
partition_kwargs=partition_config,
)

In [8]:
folds = dataloader.get_cv_folds()

Data not yet processed. Processing now...
Reading data from: C:\Users\LO79RS\topquartile\topquartile\data\covariates_may2025v2.csv
Found 134 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
THIS IS COLUMNS Index(['TOTAL_EQUITY', 'BOOK_VAL_PER_SH', 'REVENUE_PER_SH', 'RETURN_COM_EQY',
       'CUR_MKT_CAP', 'PX_LAST', 'TOT_DEBT_TO_TOT_ASSET',
       'TOT_DEBT_TO_TOT_EQY', 'BS_TOT_LIAB2', 'BS_TOT_ASSET', 'IS_EPS',
       'PX_HIGH', 'PX_LOW', 'PX_CLOSE_1D', 'PX_VOLUME', 'TURNOVER', 'ticker',
       'DVD_SH_12M'],
      dtype='object')
Applying label transformations globally to the dataset (before partitioning).
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


c:\Users\LO79RS\topquartile\topquartile\modules\datamodule\transforms\label.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.ihsg.index = pd.to_datetime(self.ihsg.index)


Data processing complete.
Partitioning data using PurgedTimeSeriesPartition for 5 splits across 85 tickers.
Fold 0: Train shape (42840, 29), Test shape (5100, 29)
Fold 1: Train shape (42840, 29), Test shape (5100, 29)
Fold 2: Train shape (42840, 29), Test shape (5100, 29)
Fold 3: Train shape (42840, 29), Test shape (5100, 29)
Fold 4: Train shape (42840, 29), Test shape (5100, 29)
Partitioning complete. Generated 5 CV folds.


In [9]:
folds = dataloader.get_cv_folds()
train_df, test_df = folds[0]
print(train_df.columns.tolist())

Partitioning data using PurgedTimeSeriesPartition for 5 splits across 85 tickers.
Fold 0: Train shape (42840, 29), Test shape (5100, 29)
Fold 1: Train shape (42840, 29), Test shape (5100, 29)
Fold 2: Train shape (42840, 29), Test shape (5100, 29)
Fold 3: Train shape (42840, 29), Test shape (5100, 29)
Fold 4: Train shape (42840, 29), Test shape (5100, 29)
Partitioning complete. Generated 5 CV folds.
['TOTAL_EQUITY', 'BOOK_VAL_PER_SH', 'REVENUE_PER_SH', 'RETURN_COM_EQY', 'CUR_MKT_CAP', 'PX_LAST', 'TOT_DEBT_TO_TOT_ASSET', 'TOT_DEBT_TO_TOT_EQY', 'BS_TOT_LIAB2', 'BS_TOT_ASSET', 'IS_EPS', 'PX_HIGH', 'PX_LOW', 'PX_CLOSE_1D', 'PX_VOLUME', 'TURNOVER', 'ticker', 'sma_20', 'sma_30', 'ema_20', 'ema_30', 'volatility_20', 'volatility_30', 'roc_126', 'momentum_change', 'eq_returns_20', 'index_returns_20', 'excess_returns_20', 'label']


In [22]:
train = folds[0][0]
test = folds[0][1]

In [23]:
train

TOTAL_EQUITY  BOOK_VAL_PER_SH  REVENUE_PER_SH  \
TickerIndex DateIndex                                                  
ADHI        2022-03-31  5582902.864        1428.8344        598.2535   
            2022-04-01  5582902.864        1428.8344        598.2535   
            2022-04-02  5582902.864        1428.8344        598.2535   
            2022-04-03  5582902.864        1428.8344        598.2535   
            2022-04-04  6022184.135        1513.8990        973.8728   
...                             ...              ...             ...   
UNVR        2024-02-27      3381238          88.6301        212.4678   
            2024-02-28      3381238          88.6301        212.4678   
            2024-02-29      3381238          88.6301        212.4678   
            2024-03-01      3381238          88.6301        212.4678   
            2024-03-04      3381238          88.6301        212.4678   

                        RETURN_COM_EQY   CUR_MKT_CAP  PX_LAST  \
TickerIndex DateIndex                                           
ADHI        2022-03-31          0.3762  2.635029e+06    678.0   
            2022-04-01          0.3762  3.275981e+06    842.0   
            2022-04-02          0.3762  2.884288e+06    742.0   
            2022-04-03          0.3762  2.724050e+06    700.0   
            2022-04-04          0.9710  2.635029e+06    678.0   
...                                ...           ...      ...   
UNVR        2024-02-27        130.1333  1.113980e+08   2920.0   
            2024-02-28        130.1333  1.068200e+08   2800.0   
            2024-02-29        130.1333           NaN      NaN   
            2024-03-01        130.1333  1.319990e+08   3460.0   
            2024-03-04        130.1333  1.045310e+08   2740.0   

                        TOT_DEBT_TO_TOT_ASSET  TOT_DEBT_TO_TOT_EQY  \
TickerIndex DateIndex                                                
ADHI        2022-03-31                28.1580             196.3527   
            2022-04-01                28.1580             196.3527   
            2022-04-02                28.1580             196.3527   
            2022-04-03                28.1580             196.3527   
            2022-04-04                27.0484             174.8732   
...                                       ...                  ...   
UNVR        2024-02-27                 4.5054              22.2044   
            2024-02-28                 4.5054              22.2044   
            2024-02-29                 4.5054              22.2044   
            2024-03-01                 4.5054              22.2044   
            2024-03-04                 4.5054              22.2044   

                        BS_TOT_LIAB2  BS_TOT_ASSET  ...       ema_20  \
TickerIndex DateIndex                               ...                
ADHI        2022-03-31   33348099.57   38931002.43  ...   698.210249   
            2022-04-01   33348099.57   38931002.43  ...   711.904511   
            2022-04-02   33348099.57   38931002.43  ...   714.770748   
            2022-04-03   33348099.57   38931002.43  ...   713.364010   
            2022-04-04   32912410.85   38934594.98  ...   709.996009   
...                              ...           ...  ...          ...   
UNVR        2024-02-27   13282848.00   16664086.00  ...  3041.139341   
            2024-02-28   13282848.00   16664086.00  ...  3018.173690   
            2024-02-29   13282848.00   16664086.00  ...  3018.173690   
            2024-03-01   13282848.00   16664086.00  ...  3064.220104   
            2024-03-04   13282848.00   16664086.00  ...  3033.341999   

                             ema_30  volatility_20  volatility_30    roc_126  \
TickerIndex DateIndex                                                          
ADHI        2022-03-31   712.087594       0.114333       0.098490 -46.529968   
            2022-04-01   720.469040       0.125182       0.108159 -19.656489   
            2022-04-02   721.858134       0.126538       0.110384 -16.90

In [10]:
train_df, test_df = folds[0]

train_df = train_df.dropna()
test_df = test_df.dropna()

y_train = train_df["label"]
y_test = test_df["label"]

X_train = train_df.drop(columns=["label"])
X_test = test_df.drop(columns=["label"])

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (5919, 28)
Test shape: (240, 28)


In [11]:
X_train.dtypes.value_counts()

float64     26
object       1
category     1
Name: count, dtype: int64

In [12]:
X_train.select_dtypes(include=["object"]).columns

Index(['TOTAL_EQUITY'], dtype='object')

In [13]:
X_train = X_train.drop(columns=["TOTAL_EQUITY"])
X_test = X_test.drop(columns=["TOTAL_EQUITY"])

In [14]:
X_train.dtypes.value_counts()

float64     26
category     1
Name: count, dtype: int64

In [15]:
X_train.select_dtypes(include=["category"]).columns

Index(['ticker'], dtype='object')

In [16]:
X_train = X_train.drop(columns=["ticker"])
X_test = X_test.drop(columns=["ticker"])

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)

print(f"Model accuracy on test set: {acc:.4f}")

Model accuracy on test set: 1.0000


In [18]:
DROP_COLS = ["label", "EXCESS_RETURN", "INDEX_RETURN", "30d_stock_return", "ticker"]
X_train = train_df.drop(columns=DROP_COLS, errors="ignore")
X_test = test_df.drop(columns=DROP_COLS, errors="ignore")

y_train = train_df["label"]
y_test = test_df["label"]

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

model.fit(X_train, y_train)

preds = model.predict(X_test)


acc = accuracy_score(y_test, preds)


print(f" Model accuracy on test set: {acc:.4f}")

 Model accuracy on test set: 1.0000


In [20]:
from quantile_forest import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def train_one_fold(fold_id: int) -> float:
    train_df, test_df = folds[fold_id]
    train_df = train_df.dropna(subset=["label"])
    test_df = test_df.dropna(subset=["label"])
    y_train = train_df["label"]
    y_test = test_df["label"]

    X_train = train_df.drop(columns=["label"])
    X_test = test_df.drop(columns=["label"])
    X_train = X_train.drop(columns=["ticker"], errors="ignore")
    X_test = X_test.drop(columns=["ticker"], errors="ignore")
    X_train = X_train.select_dtypes(exclude=["object", "category"])
    X_test = X_test.select_dtypes(exclude=["object", "category"])
    model = RandomForestQuantileRegressor(
        n_estimators=100,
        max_depth=5,
        min_samples_leaf=2,
        max_features=0.8,
        n_jobs=-1,
    )
    model.fit(X_train.values, y_train.values)
    y_pred = model.predict(X_test.values, quantiles=[0.5])
    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    return rmse

In [21]:
n_folds = 5
rmses = []

for fold_id in range(n_folds):
    print(f"Training on fold {fold_id}")
    rmse = train_one_fold(fold_id)
    print(f"Fold {fold_id} RMSE: {rmse:.4f}")
    rmses.append(rmse)

print(f"\nAverage RMSE: {np.mean(rmses):.4f}")

Training on fold 0
Fold 0 RMSE: 0.0000
Training on fold 1


KeyboardInterrupt: 